In [1]:
from src.audioProcessing import splitOnSilence

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from librosa.feature import mfcc

In [26]:
windowsList = []
mypath = './dataset/recordings/stage-1/converted'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for i,file in enumerate(files[:1]):
    fileID = int(re.findall('[0-9]+.',file)[0][:-1])
    species = re.findall('\w+-\w+_',file)[0][:-1]
    chunks = splitOnSilence(f'{mypath}/{file}')
    print(f'Splitting {file}: file {i+1} out of {len(files)}')
    for chunk in chunks:
        # Define windows with overlap:
        windowLen = 1000
        overlap = 500
        if len(chunk) >= 1000:
            for i in range(0,len(chunk),overlap):
                if i+windowLen <= len(chunk):
                    window = chunk[i:i+windowLen]
                    # Get array from each window:
                    # window = window.set_frame_rate(48000) # Así puedo cambiar la longitud del array
                    sample = window.get_array_of_samples()
                    # Check if window has a minimum of amplitude... (para intentar quitar las ventanas donde no hay pájaro...)
                    if np.max(sample) > 1500:
                        # Fourier:
                        fft_mod = np.abs(fft(sample,512))
                        fft_mod = fft_mod[0:len(fft_mod)//2]
                        mels = np.mean(mfcc(y=np.array([float(e) for e in sample]), sr=len(sample), n_mfcc=128).T, axis=0)
                        # Array of dictionaries:
                        windowsList.append({'class':species,'id':fileID,'sound':sample,'fourier':fft_mod,'mfcc':mels})

Splitting Corvus-corax_168257.mp3: file 1 out of 464


In [27]:
DF = pd.DataFrame(windowsList)

In [28]:
DF.head()

,class,fourier,id,mfcc,sound
0,Corvus-corax,"[9821.0, 10546.49915146246, 10580.035209731042...",168257,"[599.0549, 206.31346, -58.641186, -27.265524, ...","[272, 336, 389, 430, 453, 458, 450, 432, 400, ..."
1,Corvus-corax,"[8188.0, 8943.65822945392, 12497.463709316875,...",168257,"[614.64923, 219.40643, -73.49711, -26.373257, ...","[-5, -6, 3, 23, 50, 79, 109, 131, 140, 137, 12..."
2,Corvus-corax,"[23393.0, 27063.29377600895, 39150.11469883942...",168257,"[610.63336, 219.55307, -74.89993, -24.576887, ...","[1714, 1625, 1473, 1262, 1003, 721, 437, 161, ..."
3,Corvus-corax,"[856.0, 2087.100699828188, 15369.105999854602,...",168257,"[613.5123, 220.59056, -77.05687, -26.014338, -...","[53, 110, 155, 196, 238, 274, 299, 315, 325, 3..."
4,Corvus-corax,"[133.0, 2829.4605915980414, 8675.302062538418,...",168257,"[603.2568, 216.3062, -70.78507, -23.194471, -1...","[-677, -524, -371, -226, -91, 37, 168, 299, 42..."


In [30]:
len(DF.iloc[0,3])

50